In [ ]:
!pip install langchain # langchain 활용
!pip install openai==0.27.8 # chatgpt api 활용
!pip install google-search-results # Google Search 결과를 가져오기 위한 라이브러리
!pip install wikipedia # Wikipedia API를 사용하기 위한 라이브러리
!pip install faiss-cpu # FAISS는 Facebook에서 만든 오픈소스 벡터 데이터베이스 라이브러리 (MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # 텍스트를 요약할 때 필요한 라이브러리
!pip install PyPDF2 # PDF 파일을 읽어오기 위한 라이브러리

In [ ]:
pip install langchain_community

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('')
os.environ["SERPAPI_API_KEY"] = userdata.get('')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('')


In [ ]:
pip show google_search_results

In [ ]:
from langchain.llms import OpenAI
import langchain_community

llm = OpenAI(model_name = 'gpt-3.5-turbo', temperature=0.9)

In [ ]:
!pip install google-search-results

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(temperature=0)

tools = load_tools(["wikipedia", "llm-math"], llm=chat)
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

print(agent.run("Google CEO와 Tesla CEO는 각각 몇살이고 이 두사람의 나이 차이는 몇 살 차이야?"))
print()
print(agent.tools)
print(agent.tools[0].description)
print(agent.tools[1].description)

In [ ]:
from langchain.document_loaders import WebBaseLoader

web_site = "위키피디아 url"

loader = WebBaseLoader(web_path=web_site)
document = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(document)

In [ ]:
print(len(docs))


In [ ]:
print(docs[10])

In [ ]:
#요약하기
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:3])

In [ ]:
# 요약
'''
YouTube is an online video sharing platform available in 193 languages,
covering various topics such as technology, services, social impact, incidents, revenue, and copyright.
It was founded in 2005 in the United States and acquired by Google in 2006.
YouTube is known as the largest video sharing and hosting site globally, allowing users to watch, upload, and share videos.


YouTube는 193개 언어로 제공되는 온라인 동영상 공유 플랫폼입니다.
기술, 서비스, 사회적 영향, 사건, 수익, 저작권 등 다양한 주제를 다룹니다.
2005년 미국에서 설립됐으며 2006년 구글에 인수됐다.
YouTube는 전 세계적으로 가장 큰 동영상 공유 및 호스팅 사이트로 알려져 있으며 사용자가 동영상을 보고,
업로드하고, 공유할 수 있습니다.
'''



In [ ]:
#embedding and vectorstore 활용, 학습시키기

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls = FAISS,
    embedding=embeddings).from_loaders([loader])

index.vectorstore.save_local("faiss_youtube")

In [ ]:
index.query("유튜브를 출시한 날짜는 언제인가요", llm=chat, verbose=True)

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss_youtube", embeddings, allow_dangerous_deserialization=True)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("유튜브는 누가 만들었나요?", llm=chat, verbose=True)
